## Parsing recipes
Need to get: 
- recipe ID, name 
- rating for recipe 

- review text 


In [1]:
import pymongo 
import boto3 #go get 'about_me' page 
from bs4 import BeautifulSoup as BS
import pickle

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
allrecipes_db = client.allrecipes
allrecipes_db.collection_names()

[u'members', u'webpages', u'community_link_pages', u'community_page']

In [3]:
members_coll = allrecipes_db.members

In [4]:
members_coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'allrecipes'), u'members')

In [129]:
# Get all the key names 
members_coll.find_one().keys()

[u'favorites_page',
 u'following_page',
 u'followers_dict',
 u'reviews_recipe_id_list',
 u'reviews_dict',
 u'reviews_page',
 u'madeits_recipe_id_list',
 u'followers_page',
 u'favorites_dict',
 u'madeit_page',
 u'following_dict',
 u'link',
 u'following_recipe_id_list',
 u'member_ID',
 u'favorites_recipe_id_list',
 u'followers_recipe_id_list',
 u'_id',
 u'recipes_page',
 u'madeits_dict']

In [130]:
allrecipes_db.command({'dbstats': 1})

{u'avgObjSize': 308482.0759956156,
 u'collections': 4,
 u'dataSize': 1688630884.0,
 u'db': u'allrecipes',
 u'indexSize': 221184.0,
 u'indexes': 5,
 u'numExtents': 0,
 u'objects': 5474,
 u'ok': 1.0,
 u'storageSize': 639676416.0,
 u'views': 0}

In [9]:
allrecipes_db.command({'collstats': 'members'})

{u'avgObjSize': 1033871,
 u'capped': False,
 u'count': 815,
 u'indexDetails': {u'_id_': {u'LSM': {u'bloom filter false positives': 0,
    u'bloom filter hits': 0,
    u'bloom filter misses': 0,
    u'bloom filter pages evicted from cache': 0,
    u'bloom filter pages read into cache': 0,
    u'bloom filters in the LSM tree': 0,
    u'chunks in the LSM tree': 0,
    u'highest merge generation in the LSM tree': 0,
    u'queries that could have benefited from a Bloom filter that did not exist': 0,
    u'sleep for LSM checkpoint throttle': 0,
    u'sleep for LSM merge throttle': 0,
    u'total size of bloom filters': 0},
   u'block-manager': {u'allocations requiring file extension': 6,
    u'blocks allocated': 16,
    u'blocks freed': 4,
    u'checkpoint size': 8192,
    u'file allocation unit size': 4096,
    u'file bytes available for reuse': 20480,
    u'file magic number': 120897,
    u'file major version number': 1,
    u'file size in bytes': 45056,
    u'minor version number': 0},
  

In [16]:
soup = BS(members_coll.find_one()['reviews_page'], 'html.parser')

In [17]:
print soup.prettify()

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <style type="text/css">
   @charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}
  </style>
  <title>
   Public Profile - Allrecipes.com
  </title>
  <meta content="Follow me on Allrecipes!" property="og:title"/>
  <meta content="Allrecipes" property="og:site_name"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="See the recipes I’m saving, making, and sharing with friends. Let’s cook together." property="og:description"/>
  <meta content="noindex, follow" name="robots"/>
  <link href="http://allrecipes.com/cook/64043/reviews/" id="canonicalUrl" rel="canonical"/>
  <meta content="http://allrecipes.com/cook/64043/" property="og:url"/>
  <link href="http://css01.media-allrecipes.com/assets/deployables/v-1.63.0.3893-betterswap/main-css

In [5]:
# Access environment variables 
import os 
AWS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY'] 

In [6]:
# Open s3 session with boto3
session = boto3.Session(aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
s3 = session.resource('s3')

In [7]:
mybucket = s3.Bucket('ohailolcat')

In [ ]:
# mybucket.download_file('aboutme2_page.p ', 'aboutme2.pkl')

In [1]:
ls

EC2_scraper.py                     ndb_to_mongo.py
__init__.py                        ndb_to_mongo.pyc
aboutme.pkl                        non-allrecipe reviews.ipynb
aboutme2.pkl                       recipe_scrape_list.ipynb
aboutme_page.ipynb                 recipe_scraper-localmachine.ipynb
favorites_page.ipynb               recipes_0to199/
followers_page.ipynb               recipes_orderedset.pkl
following_page.ipynb               reviews_page.ipynb
foodmaster                         scrape_for_reviews.py
geckodriver.log                    scrape_for_reviews.pyc
lookit.py                          temp.pkl
madeit_page.ipynb                  temp.png
mem.py                             txt_to_mongo.py
members.py                         ubuntu@54.202.196.138
memlist.txt                        usda_food_items.ipynb


In [218]:
recipe1 = pickle.load(open('recipes_0to199/recipe_10638.pkl', 'rb'))

In [219]:
type(recipe1)

unicode

In [220]:
s = BS(recipe1,"html.parser")

In [226]:
# recipe_name = 
[a.text for a in s.select("h1.recipe-summary__h1")]

[u'Cranberry Pistachio Biscotti']

In [224]:
# description = 
[a.text.strip() for a in s.select("div.submitter__description")]

[u'"The red and green make a great Christmas cookie. Have used other nuts instead of pistachios with success. If your pistachios are salted, omit the 1/4 teaspoon salt from the recipe."']

In [221]:
# ingredients_list = 
[a.text for a in s.select("span.recipe-ingred_txt")][:-1]

[u'1/4 cup light olive oil',
 u'3/4 cup white sugar',
 u'2 teaspoons vanilla extract',
 u'1/2 teaspoon almond extract',
 u'2 eggs',
 u'1 3/4 cups all-purpose flour',
 u'1/4 teaspoon salt',
 u'1 teaspoon baking powder',
 u'1/2 cup dried cranberries',
 u'1 1/2 cups pistachio nuts']

In [213]:
# (preptime, cooktime, readytime) = 
[a.attrs['datetime'] for a in s.select("div.directions--section time")]

[u'PT15M', u'PT10M', u'PT25M']

In [214]:
# directions = 
[a.text.strip() for a in s.select("ol.list-numbers.recipe-directions__list li.step")]

[u'Preheat oven to 350 degrees F (175 degrees C).',
 u'In a medium bowl, cream together the butter, white sugar and brown sugar until smooth. Stir in the peanut butter, vanilla and egg until well blended. Combine the flour, baking soda and salt; stir into the batter just until moistened. Mix in the oats and chocolate chips until evenly distributed. Drop by tablespoonfuls on to lightly greased cookie sheets.',
 u'Bake for 10 to 12 minutes in the preheated oven, until the edges start to brown. Cool on cookie sheets for about 5 minutes before transferring to wire racks to cool completely.',
 u'']

In [215]:
# servings = 
[a.text for a in s.select("span.servings-count")]

[u'18 servings']

In [216]:
# (rating, total_stars, _, total_reviews) = 
[a.attrs['content'] for a in s.select("section.recipe-summary.clearfix meta")]

[u'4.69067239761353', u'5', u'4.69', u'1802']

In [217]:
# nutrition_label = 
[a.text.strip() for a in s.select("section.recipe-ingredients.ng-scope")]

[u'Added to shopping list.\nGo to shopping list.\n\nIngredients\n\n\n25 m\n\n\n\n\n18 servings\n\n\n\n207  cals\n\n\n\n\nfor serving adjustment\nServing size has been adjusted!\nOriginal recipe yields 18 servings\n\n\n\nAdjust\n\n\n\n\n\nUS\n\n\n\n\n\nMetric\n\n\nNote:  Recipe directions are for original size.\n\n\n\nNutrition\nAmount per serving (18 total)\n\nCalories:\n207 kcal\n\n\n\n\n\n10%\n\n\nFat: \n12 g\n\n\n\n\n\n18%\n\n\nCarbs: \n23.7g\n\n\n\n\n\n8%\n\n\nProtein: \n3.6 g\n\n\n\n\n\n7%\n\n\nCholesterol: \n24 mg\n\n\n\n\n\n8%\n\n\nSodium: \n178 mg\n\n\n\n\n\n7%\n\nBased on a 2,000 calorie diet\nSee full nutrition\n\n\n\n\nNutritional Information\nOutrageous Chocolate Chip Cookies\n\n                    1 Serving \n                    Servings Per Recipe: 18\nAmount Per Serving\n **  :    % \n **  :    % \n\n\n% Daily Value *\n\n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n **  :    % \n\n\n ** 

In [ ]:
# nutrition_label = 
[a.text for a in s.select("section.recipe-ingredients.ng-scope")]

In [75]:
nutrition_servings = [a.text.strip().split("\n") for a in s.select("div.fullNutrition div.top.light-underline")]

In [82]:
[a. for a in s.select('ul.nutrDetList div.nutrition-line-container span')]

[[u'** '],
 [],
 [u':'],
 [u' ',
  <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
  u' '],
 [],
 [u'%'],
 [],
 [u'** '],
 [],
 [u':'],
 [u' ',
  <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
  u' '],
 [],
 [u'%'],
 [],
 [u'** '],
 [],
 [u':'],
 [u' ',
  <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
  u' '],
 [],
 [u'%'],
 [],
 [u'** '],
 [],
 [u':'],
 [u' ',
  <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
  u' '],
 [],
 [u'%'],
 [],
 [u'** '],
 [],
 [u':'],
 [u' ',
  <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunit

In [188]:
# nutrition_info = 
"".join([a.text for a in s.select("div.recipe-nutrition__form li")]).split('\n')

[u'Calories:207 kcal',
 u'',
 u'',
 u'',
 u'10%Fat: 12 g',
 u'',
 u'',
 u'',
 u'18%Carbs: 23.7g',
 u'',
 u'',
 u'',
 u'8%Protein: 3.6 g',
 u'',
 u'',
 u'',
 u'7%Cholesterol: 24 mg',
 u'',
 u'',
 u'',
 u'8%Sodium: 178 mg',
 u'',
 u'',
 u'',
 u'7%',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u'*',
 u'Percent Daily Values are based on a 2,000 calorie diet. Your daily values may be higher or lower depending on your calorie needs.',
 u'',
 u'**',
 u'Nutrient information is not available for all ingredients. Amount is based on available nutrient data.',
 u'',
 u'(-)',
 u'Information is not currently available

In [190]:
# nutrition_elements = 
[a.attrs['nutrient'] for a in s.select('li.light-underline div')]

[u'::nutrition.fat',
 u'::nutrition.saturatedFat',
 u'::nutrition.cholesterol',
 u'::nutrition.sodium',
 u'::nutrition.carbohydrates',
 u'::nutrition.sugars',
 u'::nutrition.fiber',
 u'::nutrition.protein',
 u'::nutrition.vitaminA',
 u'::nutrition.vitaminC',
 u'::nutrition.calcium',
 u'::nutrition.iron',
 u'::nutrition.potassium',
 u'::nutrition.thiamin',
 u'::nutrition.niacin',
 u'::nutrition.vitaminB6',
 u'::nutrition.magnesium']

In [169]:
[a for a in s.select('li.light-underline div span.ng-binding')]

[<span class="ng-binding bold" ng-bind="::nutrient.name" ng-class="::(plainname==='true' ? 'plain' : 'bold')"></span>,
 <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
 <span class="pull-right ng-binding ng-hide" ng-bind="::nutrient.percentDailyValue" ng-hide="::(hidepercent || nutrient.percentDailyValue==='0')"></span>,
 <span class="ng-binding bold" ng-bind="::nutrient.name" ng-class="::(plainname==='true' ? 'plain' : 'bold')"></span>,
 <span class="ng-binding plain" ng-bind="nutrient.displayValue + (hideunits ? '' : nutrient.unit)" ng-class="::(boldvalue==='true' ? 'bold' : 'plain')"></span>,
 <span class="pull-right ng-binding ng-hide" ng-bind="::nutrient.percentDailyValue" ng-hide="::(hidepercent || nutrient.percentDailyValue==='0')"></span>,
 <span class="ng-binding bold" ng-bind="::nutrient.name" ng-class="::(plainname==='true' ? 'plain' : 'bold')"></span>,
 <span clas

In [227]:
recipes_coll = allrecipes_db['recipes']

In [229]:
recs = list(recipes_coll.find())

In [235]:
s = BS(recs[5]['page'], 'html.parser')

In [236]:
# nutrition_elements = 
[a.attrs['nutrient'] for a in s.select('li.light-underline div')]

[u'::nutrition.fat',
 u'::nutrition.saturatedFat',
 u'::nutrition.cholesterol',
 u'::nutrition.sodium',
 u'::nutrition.carbohydrates',
 u'::nutrition.sugars',
 u'::nutrition.fiber',
 u'::nutrition.protein',
 u'::nutrition.vitaminA',
 u'::nutrition.vitaminC',
 u'::nutrition.calcium',
 u'::nutrition.iron',
 u'::nutrition.potassium',
 u'::nutrition.thiamin',
 u'::nutrition.niacin',
 u'::nutrition.vitaminB6',
 u'::nutrition.magnesium']

In [237]:
# nutrition_info = 
"".join([a.text for a in s.select("div.recipe-nutrition__form li")]).split('\n')

[u'Calories:121 kcal',
 u'',
 u'',
 u'',
 u'6%Fat: 5.4 g',
 u'',
 u'',
 u'',
 u'8%Carbs: 17.5g',
 u'',
 u'',
 u'',
 u'6%Protein: 1.1 g',
 u'',
 u'',
 u'',
 u'2%Cholesterol: 6 mg',
 u'',
 u'',
 u'',
 u'2%Sodium: 126 mg',
 u'',
 u'',
 u'',
 u'5%',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u' **  :    % ',
 u'',
 u'*',
 u'Percent Daily Values are based on a 2,000 calorie diet. Your daily values may be higher or lower depending on your calorie needs.',
 u'',
 u'**',
 u'Nutrient information is not available for all ingredients. Amount is based on available nutrient data.',
 u'',
 u'(-)',
 u'Information is not currently available f

In [238]:
# recipe_name = 
[a.text for a in s.select("h1.recipe-summary__h1")]

[u"Grandma's Gingersnap Cookies"]